<div class="alert alert-block" style="border: 1px solid #FFB300;background-color:#F9FBE7;">
<font size="4em" style="font-weight:bold;color:#3f8dbf;">탐색적 데이터 분석</font><br>

### 전체 판매 프로세스
1. 해당 쇼핑몰에 중소업체가 계약을 맺고
2. 중소업체가 해당 쇼핑몰에 직접 상품을 올리고
2. 고객이 구매하면, 중소업체가 Olist가 제공하는 물류 파트너를 활용해서 배송을 하고,
3. 고객이 상품을 받으면, 고객에게 이메일 survey 가 전송되고,
4. 고객이 이메일 survey 에 별점과 커멘트를 남겨서 제출하게 됨
    
### 데이터 출처
- 브라질에서 가장 큰 백화점의 이커머스 쇼핑몰 (https://olist.com/)
  - 2016년도부터 2018년도 9월까지의 100k 개의 구매 데이터 정보
  - 구매 상태, 가격, 지불수단, 물류 관련, 리뷰관련, 상품 정보, 구매자 지역 관련 정보
</div>

### [ 1. 카테고리별 판매 분석 ] 

In [12]:
import pandas as pd
path = './olist_ecommerce_data/'
products = pd.read_csv( path + 'olist_products_dataset.csv', encoding = 'utf-8-sig')
customers = pd.read_csv( path + 'olist_customers_dataset.csv', encoding = 'utf-8-sig')
geolocation = pd.read_csv( path + 'olist_geolocation_dataset.csv', encoding = 'utf-8-sig')
order_items = pd.read_csv( path + 'olist_order_items_dataset.csv', encoding = 'utf-8-sig')
payments = pd.read_csv( path + 'olist_order_payments_dataset.csv', encoding = 'utf-8-sig')
reviews = pd.read_csv( path + 'olist_order_reviews_dataset.csv', encoding = 'utf-8-sig')
orders = pd.read_csv( path + 'olist_orders_dataset.csv', encoding = 'utf-8-sig')
sellers = pd.read_csv( path + 'olist_sellers_dataset.csv', encoding = 'utf-8-sig')
category_name = pd.read_csv( path + 'product_category_name_translation.csv', encoding = 'utf-8-sig')

In [13]:
products.head(1)

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0


In [14]:
category_name.head(1)

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty


#### 물품의 카테고리 구성과 물품에 관한 정보를 담은 products와 category_name 데이터를 병합

In [15]:
products_category = pd.merge(products, category_name, on = 'product_category_name')
products_category.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0,perfumery
1,6a2fb4dd53d2cdb88e0432f1284a004c,perfumaria,39.0,346.0,2.0,400.0,27.0,5.0,20.0,perfumery
2,0d009643171aee696f4733340bc2fdd0,perfumaria,52.0,150.0,1.0,422.0,21.0,16.0,18.0,perfumery
3,b1eae565a61935e0011ee7682fef9dc9,perfumaria,49.0,460.0,2.0,267.0,17.0,13.0,17.0,perfumery
4,8da90b37f0fb171b4877c124f965b1f6,perfumaria,56.0,733.0,3.0,377.0,18.0,13.0,15.0,perfumery


#### 필요 컬럼인 product_id와 product_category_name_english을 가져온다.

In [20]:
recon_products_category = products_category[['product_id', 'product_category_name_english']].copy()
recon_products_category.head()

,product_id,product_category_name_english
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumery
1,6a2fb4dd53d2cdb88e0432f1284a004c,perfumery
2,0d009643171aee696f4733340bc2fdd0,perfumery
3,b1eae565a61935e0011ee7682fef9dc9,perfumery
4,8da90b37f0fb171b4877c124f965b1f6,perfumery


In [21]:
# 결측값 확인
recon_products_category.isnull().sum()

product_id                       0
product_category_name_english    0
dtype: int64

In [22]:
# 해당상품을 구매한 주문자에 관한 자료를 가져온다.
order_items.head(1)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29


In [24]:
# 상품의 판매 트랜드를 알기위해 order_items와 recon_products_category를 병합
ordered_products_category = pd.merge(recon_products_category, order_items, on = 'product_id')
ordered_products_category.head()

,product_id,product_category_name_english,order_id,order_item_id,seller_id,shipping_limit_date,price,freight_value
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumery,e17e4f88e31525f7deef66779844ddce,1,5670f4db5b62c43d542e1b2d56b0cf7c,2018-04-30 17:33:54,10.91,7.39
1,6a2fb4dd53d2cdb88e0432f1284a004c,perfumery,048cc42e03ca8d43c729adf6962cb348,1,7040e82f899a04d1b434b795a43b4617,2017-11-23 21:31:31,16.90,7.78
2,6a2fb4dd53d2cdb88e0432f1284a004c,perfumery,5fa78e91b5cb84b6a8ccc42733f95742,1,7040e82f899a04d1b434b795a43b4617,2017-12-14 16:33:14,16.90,7.78
3,0d009643171aee696f4733340bc2fdd0,perfumery,24b1c4d88fdb7a2dc87f8ecc7d8f47f1,1,fe2032dab1a61af8794248c8196565c9,2017-08-03 09:50:14,339.00,17.13
4,0d009643171aee696f4733340bc2fdd0,perfumery,7b13c77c64a9a956500cbf1a9a23798d,1,7178f9f4dd81dcef02f62acdf8151e01,2018-05-30 10:38:35,275.00,23.48


### [  (중요,참고) 카테고리별 거래 건수 확인하기 ]
1. 카테고리 이름으로 grouping 한다: products_category_order.groupby('product_category_name_english')
2. grouping 할 때 order는 거래액이 아니라 거래건수로 count 를 한다: .aggregate({'order_id':'count'})
   - .aggregate() 또는 .agg() 를 사용
3. 거래건수로 count 한 값의 컬럼명을 order_count 로 바꾼다: .rename(columns={'order_id':'order_count'})
4. aggregate의 디폴트값은 <b>'열'</b>에 관해서 수행한다.

### aggregate 함수

In [25]:
import pandas as pd
df = pd.DataFrame({
    "미국": [2.1, 2.2, 2.3],
    "한국": [0.4, 0.5, 0.45],
    "중국": [10, 13, 15]    
})

df

,미국,한국,중국
0,2.1,0.40,10
1,2.2,0.50,13
2,2.3,0.45,15


#### 다양한 함수 사용하기
- 주요 함수: sum, min, max, mean, std, count 등

In [27]:
df.agg(['sum', 'min', 'max', 'mean', 'std', 'count'])

,미국,한국,중국
sum,6.6,1.35,38.000000
min,2.1,0.40,10.000000
max,2.3,0.50,15.000000
mean,2.2,0.45,12.666667
std,0.1,0.05,2.516611
count,3.0,3.00,3.000000


#### 특정 컬럼만 함수 적용해서 가져오기

In [29]:
# dict 형식으로 적용
df.agg({'미국' : 'sum', '한국' : 'max', '중국' : 'count'})

미국    6.6
한국    0.5
중국    3.0
dtype: float64

#### 행에 대해 함수 적용하기

In [30]:
df.agg('sum', axis = 'columns')

0    12.50
1    15.70
2    17.75
dtype: float64

#### 특정 컬럼만 grouping 하고, 함수 적용해서 가져오기

In [31]:
df = pd.DataFrame({
    "년도": [2000, 2010, 2020, 2000, 2000, 2020],    
    "미국": [2.1, 2.2, 2.3, 3.1, 4.1, 5.1],
    "한국": [0.4, 0.5, 0.45, 0.7, 0.9, 1.2],
    "중국": [10, 13, 15, 12, 45, 56]    
})

df

,년도,미국,한국,중국
0,2000,2.1,0.40,10
1,2010,2.2,0.50,13
2,2020,2.3,0.45,15
3,2000,3.1,0.70,12
4,2000,4.1,0.90,45
5,2020,5.1,1.20,56


In [36]:
# 모든 데이터프레임의 시리즈가 한가지종목에 의존하여 수립되지만,
df.groupby('년도').sum()

,미국,한국,중국
년도,,,
2000,9.3,2.00,67
2010,2.2,0.50,13
2020,7.4,1.65,71


In [35]:
# (중요)groupby에 agg를 적용할 경우, 각 데이터의 시리즈를 개별적으로 구성할 수 있다.
df.groupby('년도').agg({'미국':'sum', '한국':'mean'})

,미국,한국
년도,,
2000,9.3,0.666667
2010,2.2,0.500000
2020,7.4,0.825000


### (1) 카테고리별 거래 건수 확인하기

In [39]:
ordered_products_category.head()

,product_id,product_category_name_english,order_id,order_item_id,seller_id,shipping_limit_date,price,freight_value
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumery,e17e4f88e31525f7deef66779844ddce,1,5670f4db5b62c43d542e1b2d56b0cf7c,2018-04-30 17:33:54,10.91,7.39
1,6a2fb4dd53d2cdb88e0432f1284a004c,perfumery,048cc42e03ca8d43c729adf6962cb348,1,7040e82f899a04d1b434b795a43b4617,2017-11-23 21:31:31,16.90,7.78
2,6a2fb4dd53d2cdb88e0432f1284a004c,perfumery,5fa78e91b5cb84b6a8ccc42733f95742,1,7040e82f899a04d1b434b795a43b4617,2017-12-14 16:33:14,16.90,7.78
3,0d009643171aee696f4733340bc2fdd0,perfumery,24b1c4d88fdb7a2dc87f8ecc7d8f47f1,1,fe2032dab1a61af8794248c8196565c9,2017-08-03 09:50:14,339.00,17.13
4,0d009643171aee696f4733340bc2fdd0,perfumery,7b13c77c64a9a956500cbf1a9a23798d,1,7178f9f4dd81dcef02f62acdf8151e01,2018-05-30 10:38:35,275.00,23.48


In [40]:
ordered_products_category = ordered_products_category.groupby('product_category_name_english').agg({'order_id':'count'})
ordered_products_category.head()

,order_id
product_category_name_english,
agro_industry_and_commerce,212
air_conditioning,297
art,209
arts_and_craftmanship,24
audio,364


In [50]:
ordered_products_category = ordered_products_category.rename(columns = {'order_id':'order_count'})
ordered_products_category.head()

,order_count
product_category_name_english,
agro_industry_and_commerce,212
air_conditioning,297
art,209
arts_and_craftmanship,24
audio,364


### (2) 시각화

In [51]:
import chart_studio.plotly as py
import cufflinks as cf
cf.go_offline(connected = True)

ordered_products_category.iplot(kind = 'bar', theme = 'white')

- 범주형 자료이기 때문에 bar로 표현하는게 적합
- 출력된 현재 상태를 보면, 판매량이 적은 모든 카테고리까지 고려되어 x축 index가 난잡하며,
- 순위성을 판별하기 매우 어려움


- 위의 상황을 고려하여 graph_object 라이브러리를 이용하여 그래프를 수정한다.

In [70]:
import plotly.graph_objects as go

ordered_products_category = ordered_products_category[ordered_products_category['order_count'] >= 1000]
ordered_products_category = ordered_products_category.sort_values(by = 'order_count', ascending = False )
                                      
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x = ordered_products_category.index,
        y = ordered_products_category['order_count'],
    )
)

fig.update_layout(
    {
        'title' : {
            'text' : '<b>The number of Order per Category in Brazilian Olist E-Commerce company</b>',
            'x' : 0.5,
            'y' : 0.9,
            'font' : {
                'size' : 15
            }
        },
        
        'xaxis' : {
            'title' : '<b>from Oct. 2016 to Sep.2018</b>',
            'tickfont' : {
                'size': 10
            }
        },
        
        'yaxis' : {
            'title' : '<b>The number of Ordered</b>'
        },
        
        'template' : 'plotly_white'
    })